# Preprocessing

## Import Libraries

In [1]:
import pandas as pd
import numpy as np
import kagglehub
# import spacy
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import LancasterStemmer
from nltk.corpus import wordnet
from nltk.stem.wordnet import WordNetLemmatizer
import contractions
import re

## Download NLTK data

In [2]:
nltk.download("stopwords") # for stopword removal
nltk.download('wordnet') # for lemmatization
nltk.download('averaged_perceptron_tagger_eng') # for part-of-speech tagging

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nixosuser/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nixosuser/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /home/nixosuser/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger_eng is already up-to-
[nltk_data]       date!


True

## Load dataset

In [3]:
path = kagglehub.dataset_download("abhi8923shriv/sentiment-analysis-dataset")
train_dataset = path+"/train.csv"
test_dataset = path+"/test.csv"
print(train_dataset)
print(test_dataset)
train_df = pd.read_csv(train_dataset, encoding="ISO-8859-1")
test_df = pd.read_csv(test_dataset, encoding="ISO-8859-1")
train_df.head()

/home/nixosuser/.cache/kagglehub/datasets/abhi8923shriv/sentiment-analysis-dataset/versions/9/train.csv
/home/nixosuser/.cache/kagglehub/datasets/abhi8923shriv/sentiment-analysis-dataset/versions/9/test.csv


,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²)
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26


## Remove missing

In [4]:
train_df.dropna(inplace=True)
test_df.dropna(inplace=True)

## Expand contractions

In [7]:
# Expand contractions
def expand_contractions(text):
    text = text.replace('`', "'")
    return contractions.fix(text)

train_df["expanded_text"] = train_df["text"].apply(expand_contractions)
test_df["expanded_text"] = test_df["text"].apply(expand_contractions)

train_df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),expanded_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60,"I would have responded, if I were going"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105,Sooo SAD I will miss you here in San Diego!!!
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18,my boss is bullying me...
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164,what interview! leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26,"Sons of ****, why could not they put them on ..."


## Normalisation

In [8]:
def normalize_text(text):
    """Remove html tags, special characters, digits and convert to lowercase

    Args:
        text (str): the text to be normalized

    Returns:
        str: the normalized text
    """
    if isinstance(text, str):
        text = text.lower()
        # remove tags
        text = re.sub("&lt;/?.*?&gt;"," &lt;&gt; ", text)
        # Remove digits to avoid numbers as features
        text = re.sub(r"\d+", ' ', text)
        # remove special characters (digits already removed)
        text = re.sub(r"[^a-zA-Z\s]", '', text)
        # remove tags
        text = re.sub(r"<.*?>", '', text)
        # remove extra spaces
        text = re.sub(r"\s+", ' ', text).strip()
    else:
        text = str(text)
    return text

train_df["clean_text"] = train_df["expanded_text"].apply(normalize_text)
test_df["clean_text"] = test_df["expanded_text"].apply(normalize_text)

train_df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),expanded_text,clean_text
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60,"I would have responded, if I were going",i would have responded if i were going
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105,Sooo SAD I will miss you here in San Diego!!!,sooo sad i will miss you here in san diego
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18,my boss is bullying me...,my boss is bullying me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164,what interview! leave me alone,what interview leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26,"Sons of ****, why could not they put them on ...",sons of why could not they put them on the rel...


## Remove stopwords (CURRENTLY UNUSED)

In [9]:
def remove_stopwords(text):
    if isinstance(text, str):
        words = text.split()
        filtered_words = [word for word in words if word not in stopwords.words("english")]
        filtered_text = ' '.join(filtered_words)
        # If the whole text is just stopwords, return the original
        if len(filtered_text) == 0:
            filtered_text = text
    else:
        filtered_text = text
    return filtered_text
    
train_df["text_without_stopwords"] = train_df["clean_text"].apply(remove_stopwords)
test_df["text_without_stopwords"] = test_df["clean_text"].apply(remove_stopwords)

train_df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),expanded_text,clean_text,text_without_stopwords
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60,"I would have responded, if I were going",i would have responded if i were going,would responded going
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105,Sooo SAD I will miss you here in San Diego!!!,sooo sad i will miss you here in san diego,sooo sad miss san diego
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18,my boss is bullying me...,my boss is bullying me,boss bullying
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164,what interview! leave me alone,what interview leave me alone,interview leave alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26,"Sons of ****, why could not they put them on ...",sons of why could not they put them on the rel...,sons could put releases already bought


## Tokenization

In [10]:
def tokenize_text(text):
    try:
        text = str(text)
        tokens = word_tokenize(text)
        return tokens
    except Exception as e:
        print(f"Error tokenizing text: {e}")
        return []

train_df["tokens"] = train_df["clean_text"].apply(tokenize_text)
test_df["tokens"] = test_df["clean_text"].apply(tokenize_text)

train_df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),expanded_text,clean_text,text_without_stopwords,tokens
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60,"I would have responded, if I were going",i would have responded if i were going,would responded going,"[i, would, have, responded, if, i, were, going]"
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105,Sooo SAD I will miss you here in San Diego!!!,sooo sad i will miss you here in san diego,sooo sad miss san diego,"[sooo, sad, i, will, miss, you, here, in, san,..."
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18,my boss is bullying me...,my boss is bullying me,boss bullying,"[my, boss, is, bullying, me]"
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164,what interview! leave me alone,what interview leave me alone,interview leave alone,"[what, interview, leave, me, alone]"
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26,"Sons of ****, why could not they put them on ...",sons of why could not they put them on the rel...,sons could put releases already bought,"[sons, of, why, could, not, they, put, them, o..."


## Lemmatization with POS tagging

In [11]:
nltk_lemmatizer = WordNetLemmatizer()

def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN

def lemmatize_text(tokens):
    tokens_with_pos = nltk.pos_tag(tokens)
    lemmatized_text = []
    for token, pos in tokens_with_pos:
        wordnet_pos = get_wordnet_pos(pos)
        lemmatized_text.append(nltk_lemmatizer.lemmatize(token, pos=wordnet_pos))
    return lemmatized_text

def join_tokens(tokens):
    return ' '.join(tokens)

train_df["lemmatized_tokens"] = train_df["tokens"].apply(lemmatize_text)
test_df["lemmatized_tokens"] = test_df["tokens"].apply(lemmatize_text)
train_df["lemmatized_sentence"] = train_df["lemmatized_tokens"].apply(join_tokens)
test_df["lemmatized_sentence"] = test_df["lemmatized_tokens"].apply(join_tokens)

train_df.head()

,textID,text,selected_text,sentiment,Time of Tweet,Age of User,Country,Population -2020,Land Area (Km²),Density (P/Km²),expanded_text,clean_text,text_without_stopwords,tokens,lemmatized_tokens,lemmatized_sentence
0,cb774db0d1,"I`d have responded, if I were going","I`d have responded, if I were going",neutral,morning,0-20,Afghanistan,38928346,652860.0,60,"I would have responded, if I were going",i would have responded if i were going,would responded going,"[i, would, have, responded, if, i, were, going]","[i, would, have, respond, if, i, be, go]",i would have respond if i be go
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,Sooo SAD,negative,noon,21-30,Albania,2877797,27400.0,105,Sooo SAD I will miss you here in San Diego!!!,sooo sad i will miss you here in san diego,sooo sad miss san diego,"[sooo, sad, i, will, miss, you, here, in, san,...","[sooo, sad, i, will, miss, you, here, in, san,...",sooo sad i will miss you here in san diego
2,088c60f138,my boss is bullying me...,bullying me,negative,night,31-45,Algeria,43851044,2381740.0,18,my boss is bullying me...,my boss is bullying me,boss bullying,"[my, boss, is, bullying, me]","[my, bos, be, bully, me]",my bos be bully me
3,9642c003ef,what interview! leave me alone,leave me alone,negative,morning,46-60,Andorra,77265,470.0,164,what interview! leave me alone,what interview leave me alone,interview leave alone,"[what, interview, leave, me, alone]","[what, interview, leave, me, alone]",what interview leave me alone
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...","Sons of ****,",negative,noon,60-70,Angola,32866272,1246700.0,26,"Sons of ****, why could not they put them on ...",sons of why could not they put them on the rel...,sons could put releases already bought,"[sons, of, why, could, not, they, put, them, o...","[son, of, why, could, not, they, put, them, on...",son of why could not they put them on the rele...


## Remap sentiment

In [12]:
label_map = {
    "negative": 0,
    "neutral": 1,
    "positive": 2
}

train_df["label"] = train_df["sentiment"].map(label_map)
test_df["label"] = test_df["sentiment"].map(label_map)

## Save preprocessed data

In [13]:
train_df.to_csv("train_preprocessed.csv", index=False)
test_df.to_csv("test_preprocessed.csv", index=False)